In [1]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from IPython.display import HTML, clear_output

import cv2
import multiprocessing as mp
import sys, os, threading

from segment import road_segmentation
from ipmdistance import getDistance

plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

plt.ioff()

In [2]:
# Turn on the detection pipeline
from detect import *
from utils import convert_to_original_size

GPU found


In [3]:
# Reset previous initiation
tf.reset_default_graph()

# Instantiate Detection Object
detection_object = Detector()

INFO:tensorflow:Restoring parameters from ./saved_model/model.ckpt
Model restored in 0.626s


In [4]:
# Function to load images into memory
def load_images(path):

    getFrames = []
    for imgs in sorted(os.listdir(path)):
        getFrames.append(Image.open(os.path.join(path, imgs)))
    return getFrames

In [5]:
# Get the bounding boxes for detected objects and initiate trackers for them

def runDetection(frame):
    
    pilImage = Image.fromarray(frame)
    outputImage, bounding_boxes = detection_object.infer(pilImage)
    outputFrame = np.asarray(outputImage)
    
    # Initiate trackers for the bounding boxes
    detectedObjects = []
    results = []
    trackers = []
    
    for cls, bboxes in bounding_boxes.items():
        for box, score in bboxes:
            if(np.all(box>0)):
                detectedObjects.append(box)
    
    ntrackers = len(detectedObjects)
    for r in range(ntrackers):
        trck = cv2.TrackerMIL_create()
        x0,y0,x1,y1 = tuple(detectedObjects[r].astype(int))
        if x1-x0<=500 and y1-y0<=500:
            try:
                trck.init(frame, (x0,y0,x1-x0, y1-y0))
                trackers.append(trck)
                results.append((x0,y0,x1-x0,y1-y0))
            except:
                print('Error Encountered')
                continue
    
    return outputFrame, trackers, results
    

In [6]:
# Function to track an object in consecutive frames
def SingleTracker(trackerObject, vid_frame, output):

    try:
        ret, bbox = trackerObject.update(vid_frame)
    except:
        pass
    if(ret):
        output.put(bbox)

In [7]:
# Multiple Instance Tracking with threading in python
def parallelTracking(frame, trackers, output):
    
    threads = [threading.Thread(target=SingleTracker, args=(trck, frame, output,)) for trck in trackers]
    for t in threads: t.start()
    for t in threads: t.join()
    
    results = [output.get() for t in threads]
    for t in trackers: del t
    
    for box in results:
        p1 = (int(box[0]), int(box[1]))
        p2 = (int(box[0]+box[2]), int(box[1]+box[3]))
        cv2.rectangle(frame, p1, p2, (0,0,200), 2, 1)
    
    return frame, results

In [8]:
# Define video handle
cap = cv2.VideoCapture("input_video.mp4")

# Counter for frames
count = 0

try:
    while(True):
        ret, frame = cap.read()
        if not ret:
            cap.release()
            print("Released Video Resource")
            break
        
        H,W,_ = frame.shape
        print(count)
        
        # Run YOLO detection for every 10th frame
        if count%10==0:
            frame, trackers, results = runDetection(frame)
#             print('detected')
            output = mp.Queue()
        else:
            frame, results = parallelTracking(frame, trackers, output)
#             print('Tracked')
        
        output = mp.Queue()
        thread_distance = threading.Thread(target = getDistance, args=(frame, results, output,))
        thread_segment = threading.Thread(target = road_segmentation, args = (frame, output,))
        threads = [thread_distance, thread_segment]
        for t in threads: t.start()
        for t in threads: t.join()
        res = [output.get() for t in threads]
        for r in res:
            if type(r)==type({}):
                distance_to_objects = r
            else:
                frame = r
        
        for k in distance_to_objects.keys():
            mid_x = int(distance_to_objects[k][0]+distance_to_objects[k][2]/2)
            mid_y = int(distance_to_objects[k][1]+distance_to_objects[k][3])
            cv2.putText(frame, str(k)[:4]+' meters',(mid_x,mid_y+3), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 7, cv2.LINE_AA)

        frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        count+=1
        cv2.imshow("OUTPUT", frame)
        k = cv2.waitKey(1) & 0xff
        if(k==27):
            cv2.destroyAllWindows()
            break
        cv2.imwrite("output/frames/frame_{}.png".format(count), frame)
except KeyboardInterrupt:
    cap.release()
    clear_output()
    print("Released Video Resource due to interrupt")

0
Predictions found in 5.441s
detected


/home/ronit/Documents/BTP/Final_navigation/segment.py:13: RuntimeWarning: divide by zero encountered in log
  ii_image = 0.5 + np.log(inp_img[:,:,1]) - alpha*np.log(inp_img[:,:,2]) - (1-alpha)*np.log(inp_img[:,:,0])
/home/ronit/Documents/BTP/Final_navigation/segment.py:13: RuntimeWarning: invalid value encountered in subtract
  ii_image = 0.5 + np.log(inp_img[:,:,1]) - alpha*np.log(inp_img[:,:,2]) - (1-alpha)*np.log(inp_img[:,:,0])


1
Tracked
2
Tracked
3
Tracked
4
Tracked
5
Tracked
6
Tracked
7
Tracked
8
Tracked
9
Tracked
10
Predictions found in 0.601s
detected
11
Tracked
12
Tracked
13
Tracked
14
Tracked
15
Tracked
16
Tracked
17
Tracked
18
Tracked
19
Tracked
20
Predictions found in 0.606s
detected
21
Tracked
22
Tracked
